In [20]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry  import box

raster_path = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/embeddings/1-embeddings.tif"
gpkg_path = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tiles/1-mvp-warped.tif"

In [22]:
import rasterio
from rasterio.mask import mask
import numpy as np

# Abrir o raster que será usado como máscara
with rasterio.open(gpkg_path) as src_mask:
    mask_data = src_mask.read(1)  # lê a primeira banda
    mask_transform = src_mask.transform
    mask_crs = src_mask.crs

# Criar uma máscara booleana: True onde o valor é maior que 0 (ou qualquer critério que você queira)
boolean_mask = mask_data >= 0

# Abrir o raster que será mascarado
with rasterio.open(raster_path) as src_features:
    features_data = src_features.read()  # lê todas as bandas
    features_meta = src_features.meta.copy()

    # Ajuste da meta se necessário
    features_meta.update(dtype=rasterio.float32)

    # Aplicar a máscara banda a banda
    masked_data = np.where(boolean_mask, features_data, 0)

# Salvar o raster resultante
with rasterio.open("features_masked.tif", "w", **features_meta) as dst:
    dst.write(masked_data.astype(rasterio.float32))

In [23]:
import rasterio
import numpy as np

# Abrir o raster mascarado
with rasterio.open("features_masked.tif") as src_features:
    features_data = src_features.read()  # shape: (bandas, altura, largura)
    features_meta = src_features.meta.copy()

# Abrir o raster 1-mvc
with rasterio.open(gpkg_path) as src_mvc:
    mvc_data = src_mvc.read(1)  # lê a primeira banda

# Adicionar a banda do 1-mvc ao final do features_data
combined_data = np.vstack([features_data, mvc_data[np.newaxis, :, :]])  # shape: (65, altura, largura)

# Atualizar o metadata para refletir a nova quantidade de bandas
features_meta.update(count=combined_data.shape[0], dtype=rasterio.float32)

# Salvar o raster combinado
with rasterio.open("features_plus_mvc.tif", "w", **features_meta) as dst:
    dst.write(combined_data.astype(rasterio.float32))
